[![preview notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/devops_tests/blob/main/test_files/template.ipynb)
[![launch on mybinder.org](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/devops_tests.git/main?urlpath=lab/tree/test_files/template.ipynb)
[![launch on Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/devops_tests/blob/main/test_files/template.ipynb)

### based on Table 1 from B.Bolin 1958 "On the use of tritium as a tracer for water in nature".
$\frac{1}{M_s} \frac{d M_s}{dt} = c_1 \frac{1}{M}\frac{dM}{dt}$

$-\frac{3}{a^2}r\frac{dr}{dt}c_1$

#linki: artykuł, wikipedia

In [43]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PySDM-examples')

In [30]:
import numpy as np
import pandas
from PySDM.physics import in_unit, si
from PySDM import Formulae

In [31]:
radii = np.asarray([0.005, 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.20]) * si.cm
c1 = 1.63

In [61]:
formulae = Formulae(
    terminal_velocity='RogersYau',
    drop_growth='Mason1951',
    diffusion_thermics='Neglect',
    saturation_vapour_pressure='AugustRocheMagnus',
    ventilation='Froessling1938',
    particle_shape_and_density='LiquidSpheres',
    air_dynamic_viscosity='ZografosEtAl1987',
)
const = formulae.constants
any_non_zero_value = 44.
temperature = const.T0 + 10 * si.K
pressure = const.p_STP
pvs = formulae.saturation_vapour_pressure.pvs_water(temperature)
v_term = formulae.terminal_velocity.v_term(radii)
eta_air=formulae.air_dynamic_viscosity.eta_air(temperature)
D=formulae.diffusion_thermics.D(T=temperature, p=pressure)

air_density = pressure/const.Rd/temperature
assert abs(air_density - 1)/air_density <.3
Re = formulae.particle_shape_and_density.reynolds_number(
     radius=radii,
     velocity_wrt_air=v_term,
     dynamic_viscosity=eta_air,
     density=air_density,
)
Sc = formulae.trivia.air_schmidt_number(
    dynamic_viscosity=eta_air, 
    diffusivity=D, 
    density=air_density,
)
F = formulae.ventilation.ventilation_coefficient(sqrt_re_times_cbrt_sc=Re**(1/2) * Sc**(1/3))
f = .75

r_dr_dt = formulae.drop_growth.r_dr_dt(
    RH_eq=1,
    T=temperature,
    RH=0,
    lv=0,
    pvs=pvs,
    D=D,
    K=any_non_zero_value,
    ventilation_factor=F
)
adjustment_time = (-3 / radii**2 * r_dr_dt * c1)**-1


pandas.options.display.float_format = '{:<.3g}'.format
data = pandas.DataFrame({
    'radius [cm]': in_unit(radii, si.cm),
    'adjustment time [s]': adjustment_time,
    'terminal velocity [m/s]': v_term,
    'd [m]': v_term * adjustment_time,
    
})

data

1.2519508532126034


,radius [cm],adjustment time [s],terminal velocity [m/s],d [m]
0,0.005,1.72,0.4,0.69
1,0.01,5.37,0.8,4.29
2,0.025,20.1,2,40.3
3,0.05,48.4,4,193
4,0.075,80.7,5.5,444
5,0.1,119,6.36,757
6,0.15,204,7.78,1.59e+03
7,0.2,298,8.99,2.68e+03


In [64]:
data["radius [cm]"][0]

KeyError: ('radius [cm]', 0)